European Put Vanilla using Crank-Nicolson

In [22]:
import numpy as np
import scipy.linalg as linalg
import math


# Parameters
S0 = 100
K = 100
sigma = 0.20
r = 0.025
T = 231 / 360

# Crank-Nicolson method
def crank_nicolson_european_put(S0, K, sigma, r, T, M, N):
    dt = T / N
    dS = 2 * S0 / M
    lamb = r * dt / 2
    gamma = dt * sigma**2 / 2

    S = np.linspace(0, 2 * S0, M + 1)
    V = np.maximum(K - S, 0)
    L = np.diag((1 + gamma) * np.ones(M - 1)) + np.diag(-gamma / 2 * np.arange(2, M) * dS, 1) + np.diag(-gamma / 2 * np.arange(2, M) * dS, -1)
    R = np.diag((1 - gamma) * np.ones(M - 1)) + np.diag(gamma / 2 * np.arange(2, M) * dS, 1) + np.diag(gamma / 2 * np.arange(2, M) * dS, -1)
    
    for _ in range(N):
        V[1:-1] = linalg.solve(L, R @ V[1:-1])
        V[-1] = 2 * V[-2] - V[-3]
        V[0] = 2 * V[1] - V[2]
    
    return np.interp(S0, S, V)

# Pricing
M = 300
N = 200 
price = crank_nicolson_european_put(S0, K, sigma, r, T, M, N)
print(f"European Put Vanilla price: {price}")


European Put Vanilla price: 5.042736866186616


In [24]:
T = 231 / 360 #period of contract
S_0 = 100  #price at time zero
K = 100  #exercise price
sigma = 0.2  #Volatility
r = 0.025 #Risk-neutral interest-rate
price = 0  #Just initialization :)

S_max = 100

N = 500
M = 50
dt = T / N
ds = S_max / M

f = np.zeros((M+1,N+1))  # The array f is the mesh of approximation of the option price function
I = np.arange(0, M+1)
J = np.arange(0, N+1)

# Boundary and final conditions
f[:, N] = np.maximum(K - (I * ds), 0)
f[0, :] = K * np.exp(-r * (T - J * dt))
f[M, :] = 0

alpha = 0.25 * dt * (sigma**2 * (I**2) - r * I)
beta = -dt * 0.5 * (sigma**2 * (I**2) + r)
gamma = 0.25 * dt * (sigma**2 * (I**2) + r * I)

M1 = np.diag(1-beta[1:M]) + np.diag(-alpha[2:M], k=-1) + np.diag(-gamma[1:M-1], k=1)
M2 = np.diag(1+beta[1:M]) + np.diag(alpha[2:M], k=-1) + np.diag(gamma[1:M-1], k=1)

for j in range(N-1, -1, -1):
    l = np.zeros(M - 1)
    l[0] = alpha[1] * (f[0, j] + f[0, j+1])
    l[-1] = gamma[M-1] * (f[M, j] + f[M, j+1])
    f[1:M, j] = np.linalg.solve(M1, M2 @ f[1:M, j+1] + l)

## Finding the price by interapolation
idown = int(np.floor(S_0 / ds))
iup = int(np.ceil(S_0 / ds))
print(idown)
print(iup)
if idown == iup:
    price = f[idown, 0]
else:
    price = f[idown, 0] + ((iup - (S_0 / ds)) / (iup - idown)) * (f[iup, 0] - f[idown, 0])

print(price)

50
50
0.0


In [50]:
# Crank-Nicolson with PSOR for American Put
def crank_nicolson_american_put(S0, K, sigma, r, T, M, N, omega=1.5, tol=1e-8):
    dt = T / N
    dS = 2 * S0 / M
    lamb = r * dt / 2
    gamma = dt * sigma**2 / 2

    S = np.linspace(0, 2 * S0, M + 1)
    V = np.maximum(K - S, 0)
    L = np.diag((1 + gamma) * np.ones(M - 1)) + np.diag(-gamma / 2 * np.arange(2, M) * dS, 1) + np.diag(-gamma / 2 * np.arange(2, M) * dS, -1)
    R = np.diag((1 - gamma) * np.ones(M - 1)) + np.diag(gamma / 2 * np.arange(2, M) * dS, 1) + np.diag(gamma / 2 * np.arange(2, M) * dS, -1)
    
    for _ in range(N):
        V_old = V.copy()
        V[1:-1] = linalg.solve(L, R @ V[1:-1])
        V[-1] = 2 * V[-2] - V[-3]
        V[0] = 2 * V[1] - V[2]

        # Projected SOR for early exercise
        for _ in range(1000):
            V_new = V.copy()
            for j in range(1, M):
                V_new[j] = max(V_old[j], (1 - omega) * V[j] + omega / (1 + lamb) * (V[j - 1] + lamb * V[j + 1] - gamma * j * dS * (V[j + 1] - V[j - 1])))
            
            if np.linalg.norm(V_new - V) < tol:
                break
            V = V_new
    
    return np.interp(S0, S, V)

# Pricing
M = 100
N = 100
price_american = crank_nicolson_american_put(S0, K, sigma, r, T, M, N)
print(f"American Put Vanilla price: {price_american}")

American Put Vanilla price: 99.95020163550515


To price a continuous down and in put barrier option, you can use the analytical formula for barrier options. The formula for a down-and-in put barrier option is:

Price = EuropeanPut - DownOutPut

where EuropeanPut is the price of the European put option and DownOutPut is the price of the down-and-out put barrier option. We have already calculated the European put option price in part (a). Now we need to calculate the price of the down-and-out put barrier option.

First, we will use the Black-Scholes formula to calculate the price of the down-and-out put barrier option:

In [38]:
from scipy.stats import norm

def black_scholes_put(S, K, r, sigma, T):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

def down_out_put_barrier(S0, K, r, sigma, T, barrier):
    if S0 <= barrier:
        return 0

    lam = (r + 0.5 * sigma**2) / (sigma**2)
    y = np.log(barrier**2 / (S0 * K)) / (sigma * np.sqrt(T)) + lam * sigma * np.sqrt(T)
    x1 = np.log(S0 / barrier) / (sigma * np.sqrt(T)) + lam * sigma * np.sqrt(T)
    x2 = np.log(barrier / S0) / (sigma * np.sqrt(T)) + lam * sigma * np.sqrt(T)

    A = black_scholes_put(S0, K, r, sigma, T)
    B = (barrier / S0)**(2 * lam) * black_scholes_put(S0, K * (barrier / S0)**(2 - 2 * lam), r, sigma, T)
    C = -norm.cdf(x1) * S0 * np.exp(-r * T) * ((barrier / S0)**(2 * lam) - 1) / (2 * lam - 2)
    D = -K * np.exp(-r * T) * (norm.cdf(y) - (barrier / S0)**(2 * lam - 2) * norm.cdf(y - 2 * lam * sigma * np.sqrt(T))) / (2 * lam - 2)
    
    return A - B + C + D

barrier = 90
price_down_in_put = price - down_out_put_barrier(S0, K, r, sigma, T, barrier)
print(f"Continuous Down and In Put Barrier Option price: {price_down_in_put}")


Continuous Down and In Put Barrier Option price: 54.881459150322456


In [40]:
# Black-Scholes for European Put
european_put_bs = black_scholes_put(S0, K, r, sigma, T)
print(f"Black-Scholes European Put price: {european_put_bs}")
print(f"PDE European Put price: {price}")
print(f"Absolute difference: {abs(european_put_bs - price)}")

# Black-Scholes for Down-and-In Put Barrier
down_in_put_bs = european_put_bs - down_out_put_barrier(S0, K, r, sigma, T, barrier)
print(f"Black-Scholes Down and In Put Barrier price: {down_in_put_bs}")
print(f"PDE Down and In Put Barrier price: {price_down_in_put}")
print(f"Absolute difference: {abs(down_in_put_bs - price_down_in_put)}")

# Trinomial Tree for American Put
def trinomial_tree_american_put(S0, K, r, sigma, T, N):
    dt = T / N
    u = np.exp(sigma * np.sqrt(2 * dt))
    d = 1 / u
    m = 1
    p_u= ((np.exp(r * dt / 2) - np.exp(-sigma * np.sqrt(dt / 2))) / (np.exp(sigma * np.sqrt(dt / 2)) - np.exp(-sigma * np.sqrt(dt / 2))))**2
    p_d = ((np.exp(sigma * np.sqrt(dt / 2)) - np.exp(r * dt / 2)) / (np.exp(sigma * np.sqrt(dt / 2)) - np.exp(-sigma * np.sqrt(dt / 2))))**2
    p_m = 1 - p_u - p_d
    
    S = np.zeros((N + 1, N + 1))
    S[:, 0] = S0
    for j in range(1, N + 1):
        S[1:-1, j] = S[0:-2, j - 1] * u + S[2:, j - 1] * d
    
    V = np.maximum(K - S[:, -1], 0)
    for j in range(N - 1, -1, -1):
        V[:-2] = np.exp(-r * dt) * (p_u * V[:-2] + p_m * V[1:-1] + p_d * V[2:])
        V = np.maximum(K - S[:, j], V)
    
    return V[N]

american_put_trinomial_tree = trinomial_tree_american_put(S0, K, r, sigma, T, N)
print(f"Trinomial Tree American Put price: {american_put_trinomial_tree}")
print(f"PDE American Put price: {price_american}")
print(f"Absolute difference: {abs(american_put_trinomial_tree - price_american)}")


Black-Scholes European Put price: 5.569724706320073
PDE European Put price: 96.7282712611355
Absolute difference: 91.15854655481543
Black-Scholes Down and In Put Barrier price: -36.277087404492974
PDE Down and In Put Barrier price: 54.881459150322456
Absolute difference: 91.15854655481543
Trinomial Tree American Put price: 100.0
PDE American Put price: 99.4167213091466
Absolute difference: 0.5832786908534047


In [44]:
import pandas as pd

def monte_carlo_european_put(S0, K, r, sigma, T, num_simulations):
    np.random.seed(42)  # For reproducibility
    num_timesteps = 10
    dt = T / num_timesteps

    total_payoff = 0
    for i in range(num_simulations):
        S = S0
        for j in range(num_timesteps):
            z = np.random.standard_normal()
            S *= np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)

        total_payoff += max(K - S, 0)

    price = np.exp(-r * T) * total_payoff / num_simulations
    return price

# Parameters from part (a)
S0 = 100
K = 100
r = 0.025
sigma = 0.20


num_simulations = 10000
monte_carlo_price = monte_carlo_european_put(S0, K, r, sigma, T, num_simulations)

print(f"Monte Carlo European Put price: {monte_carlo_price}")
print(f"PDE European Put price: {price}")  # Assuming 'price' is the PDE price from part (a)
print(f"Absolute difference: {abs(monte_carlo_price - price)}")



Monte Carlo European Put price: 5.667516673741924
PDE European Put price: 96.7282712611355
Absolute difference: 91.06075458739357
